<a href="https://colab.research.google.com/github/extrajp2014/DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments/blob/master/DS_Unit_1_Sprint_Challenge_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 1 Sprint Challenge 4

## Exploring Data, Testing Hypotheses

In this sprint challenge you will look at a dataset of people being approved or rejected for credit.

https://archive.ics.uci.edu/ml/datasets/Credit+Approval

Data Set Information: This file concerns credit card applications. All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.

Attribute Information:
- A1: b, a.
- A2: continuous.
- A3: continuous.
- A4: u, y, l, t.
- A5: g, p, gg.
- A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
- A7: v, h, bb, j, n, z, dd, ff, o.
- A8: continuous.
- A9: t, f.
- A10: t, f.
- A11: continuous.
- A12: t, f.
- A13: g, p, s.
- A14: continuous.
- A15: continuous.
- A16: +,- (class attribute)

Yes, most of that doesn't mean anything. A16 (the class attribute) is the most interesting, as it separates the 307 approved cases from the 383 rejected cases. The remaining variables have been obfuscated for privacy - a challenge you may have to deal with in your data science career.

Sprint challenges are evaluated based on satisfactory completion of each part. It is suggested you work through it in order, getting each aspect reasonably working, before trying to deeply explore, iterate, or refine any given step. Once you get to the end, if you want to go back and improve things, go for it!

## Part 1 - Load and validate the data

- Load the data as a `pandas` data frame.
- Validate that it has the appropriate number of observations (you can check the raw file, and also read the dataset description from UCI).
- UCI says there should be missing data - check, and if necessary change the data so pandas recognizes it as na
- Make sure that the loaded features are of the types described above (continuous values should be treated as float), and correct as necessary

This is review, but skills that you'll use at the start of any data exploration. Further, you may have to do some investigation to figure out which file to load from - that is part of the puzzle.

In [1]:
url="https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data"
!wget $url
!curl $url | wc

--2019-02-01 16:06:54--  https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.249
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32218 (31K) [text/plain]
Saving to: ‘crx.data’

crx.data            100%[===================>]  31.46K  --.-KB/s    in 0.03s   

2019-02-01 16:06:59 (1004 KB/s) - ‘crx.data’ saved [32218/32218]

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 32218  100 32218    0     0   154k      0 --:--:-- --:--:-- --:--:--  153k
    690     690   32218


In [2]:
# load data
from scipy.stats import chisquare
from scipy import stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.height', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 500)
pd.options.display.float_format = '{:,}'.format

names=["A"+str(i) for i in range(1,17)]
df = pd.read_csv('crx.data', header=None, names=names)
df.head()


,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.0,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.46,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.5,u,g,q,h,1.5,t,f,0,f,g,00280,824,+
3,b,27.83,1.54,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [42]:
# preview data
print("df shape:"), print(df.shape), print("")
print("df dtypes:"), print(df.dtypes), print("")
print("df sample(7):"), print(df.sample(7)), print("")
print("df isnull().sum().sum():"), print(df.isnull().sum().sum()), print("")
print("df describe(include=np.number):")
print(df.describe(include=np.number))
print("")
print("df describe(exclude=np.number):")
print(df.describe(exclude=np.number))
print("")

df shape:
(690, 16)

df dtypes:
A1      object
A2     float64
A3     float64
A4      object
A5      object
A6      object
A7      object
A8     float64
A9      object
A10     object
A11    float64
A12     object
A13     object
A14    float64
A15    float64
A16     object
dtype: object

df sample(7):
    A1    A2    A3 A4 A5  A6  A7    A8 A9 A10  A11 A12 A13   A14     A15 A16
662  a  23.5   1.5  u  g   w   v 0.875  f   f  0.0   t   g 160.0     0.0   -
280  b 21.17 0.875  y  p   c   h  0.25  f   f  0.0   f   g 280.0   204.0   -
347  b 26.75   4.5  y  p   c  bb   2.5  f   f  0.0   f   g 200.0 1,210.0   -
634  a 23.75  0.71  u  g   w   v  0.25  f   t  1.0   t   g 240.0     4.0   -
229  b 22.08  11.0  u  g  cc   v 0.665  t   f  0.0   f   g 100.0     0.0   +
547  b 23.92   1.5  u  g   d   h 1.875  t   t  6.0   f   g 200.0   327.0   +
194  b  34.5  4.04  y  p   i  bb   8.5  t   t  7.0   t   g 195.0     0.0   +

df isnull().sum().sum():
67

df describe(include=np.number):
                     

In [0]:
# Documentation: https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.names
# A1: b, a.
# A2: continuous.
# A3: continuous.
# A4: u, y, l, t.
# A5: g, p, gg.
# A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
# A7: v, h, bb, j, n, z, dd, ff, o.
# A8: continuous.
# A9: t, f.
# A10: t, f.
# A11: continuous.
# A12: t, f.
# A13: g, p, s.
# A14: continuous.
# A15: continuous.
# A16: +,- (class attribute)

# 8.  Missing Attribute Values:
#     37 cases (5%) have one or more missing values.  The missing
#     values from particular attributes are:

#     A1:  12
#     A2:  12
#     A4:   6
#     A5:   6
#     A6:   9
#     A7:   9
#     A14: 13

In [48]:
# UCI says there should be missing data - check, and if necessary change the data 
# so pandas recognizes it as na
# Make sure that the loaded features are of the types described above (continuous 
# values should be treated as float), and correct as necessary

df = df.replace({'?': np.nan})
# convert to correct format
for i in ["A2","A3","A8","A11","A14","A15"]:
  df[i] = df[i].astype(float)

# preview data
df2 = df.dropna()
print(df.shape, df2.shape), print("")
print(df2['A16'].value_counts()), print("")
print("df isnull().sum():"), print(df.isnull().sum()), print("")
print("df2 isnull().sum().sum():"), print(df2.isnull().sum().sum()), print("")
print("df2 sample(7):"), print(df2.sample(7)), print("")
print("df2 describe(include=np.number):")
print(df2.describe(include=np.number))
print("")
print("df2 describe(exclude=np.number):")
print(df2.describe(exclude=np.number))
print("")

(690, 16) (653, 16)

-    357
+    296
Name: A16, dtype: int64

df isnull().sum():
A1     12
A2     12
A3      0
A4      6
A5      6
A6      9
A7      9
A8      0
A9      0
A10     0
A11     0
A12     0
A13     0
A14    13
A15     0
A16     0
dtype: int64

df2 isnull().sum().sum():
0

df2 sample(7):
    A1    A2    A3 A4 A5  A6 A7    A8 A9 A10  A11 A12 A13   A14     A15 A16
285  b 17.58  10.0  u  g   w  h 0.165  f   t  1.0   f   g 120.0     1.0   -
11   b 29.92 1.835  u  g   c  h 4.335  t   f  0.0   f   g 260.0   200.0   +
220  a 50.08 12.54  u  g  aa  v  2.29  t   t  3.0   t   g 156.0     0.0   +
166  b 19.33   9.5  u  g   q  v   1.0  t   f  0.0   t   g  60.0   400.0   +
429  b 33.58 0.335  y  p  cc  v 0.085  f   f  0.0   f   g 180.0     0.0   -
510  b 13.75   4.0  y  p   w  v  1.75  t   t  2.0   t   g 120.0 1,000.0   +
28   b 57.42   8.5  u  g   e  h   7.0  t   t  3.0   f   g   0.0     0.0   +

df2 describe(include=np.number):
                      A2                A3               

## Part 2 - Exploring data, Testing hypotheses

The only thing we really know about this data is that A16 is the class label. Besides that, we have 6 continuous (float) features and 9 categorical features.

Explore the data: you can use whatever approach (tables, utility functions, visualizations) to get an impression of the distributions and relationships of the variables. In general, your goal is to understand how the features are different when grouped by the two class labels (`+` and `-`).

For the 6 continuous features, how are they different when split between the two class labels? Choose two features to run t-tests (again split by class label) - specifically, select one feature that is *extremely* different between the classes, and another feature that is notably less different (though perhaps still "statistically significantly" different). You may have to explore more than two features to do this.

For the categorical features, explore by creating "cross tabs" (aka [contingency tables](https://en.wikipedia.org/wiki/Contingency_table)) between them and the class label, and apply the Chi-squared test to them. [pandas.crosstab](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html) can create contingency tables, and [scipy.stats.chi2_contingency](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html) can calculate the Chi-squared statistic for them.

There are 9 categorical features - as with the t-test, try to find one where the Chi-squared test returns an extreme result (rejecting the null that the data are independent), and one where it is less extreme.

**NOTE** - "less extreme" just means smaller test statistic/larger p-value. Even the least extreme differences may be strongly statistically significant.

Your *main* goal is the hypothesis tests, so don't spend too much time on the exploration/visualization piece. That is just a means to an end - use simple visualizations, such as boxplots or a scatter matrix (both built in to pandas), to get a feel for the overall distribution of the variables.

This is challenging, so manage your time and aim for a baseline of at least running two t-tests and two Chi-squared tests before polishing. And don't forget to answer the questions in part 3, even if your results in this part aren't what you want them to be.

In [10]:
# Split class into 2 df for stats calculation
pos_df = df2[df2['A16']=='+']
neg_df = df2[df2['A16']=='-']
print(pos_df.shape,neg_df.shape)

(296, 16) (357, 16)


In [43]:
floatColumns=['A2','A3','A8','A11','A14','A15']
categoricColumns=['A1','A4','A5','A6','A7','A9','A10','A12','A13']
boolColumns=['A9','A10','A12']

# Calculate 2 SAMPLE TTEST - Nonequal sample sizes
print("Pvalue < 0.05")
print("Column","       ","Statistic","              ", "Pvalue")
for i in floatColumns:
  stat, pvalue = stats.ttest_ind(pos_df[i], neg_df[i], nan_policy='omit', equal_var=False)
  if pvalue < 0.05: print(i,"       ", stat,"       ", pvalue)
print("")

print("Pvalue > 0.05")
print("Column","       ","Statistic","              ", "Pvalue")
for i in floatColumns:
  stat, pvalue = stats.ttest_ind(pos_df[i], neg_df[i], nan_policy='omit', equal_var=False)
  if pvalue > 0.05: print(i,"       ", stat,"       ", pvalue)
print("")

Pvalue < 0.05
Column         Statistic                Pvalue
A2         4.603105537295886         5.118622270751413e-06
A3         5.290896422208584         1.7489090249152318e-07
A8         8.497111846543877         3.6540467526389407e-16
A11         10.4894892779921         1.702907212870187e-22
A14         -2.193851777301385         0.02860488893016181
A15         4.079693539983498         5.792817047242567e-05

Pvalue > 0.05
Column         Statistic                Pvalue



In [17]:
# Preview data for chi-square test
for i in categoricColumns:
  print(df2[i].value_counts())

b    450
a    203
Name: A1, dtype: int64
u    499
y    152
l      2
Name: A4, dtype: int64
g     499
p     152
gg      2
Name: A5, dtype: int64
c     133
q      75
w      63
i      55
aa     52
ff     50
k      48
cc     40
m      38
x      36
d      26
e      24
j      10
r       3
Name: A6, dtype: int64
v     381
h     137
ff     54
bb     53
z       8
j       8
dd      6
n       4
o       2
Name: A7, dtype: int64
t    349
f    304
Name: A9, dtype: int64
f    366
t    287
Name: A10, dtype: int64
f    351
t    302
Name: A12, dtype: int64
g    598
s     53
p      2
Name: A13, dtype: int64


In [47]:
# Get significant Chi-Square crosstab
def significantChi2(oneColumn,columns,df):
  '''
  For p-value < 0.05, print Pearson chisquare test results of 1 column cross with 
  all other categoric columns from chosen columns
  '''
  for i in columns:
    temp=pd.crosstab(df[oneColumn], df[i])
    if stats.chi2_contingency(temp)[1] < .05:
      print("---------------------")
      print(temp)
      print("---------------------")
      print("Chisquare of A16 crosstab", i,":")
      print("Chi2 Statistic,", "P-value,", "Degree of Freedom,", "Expected Frequency")
      print(stats.chi2_contingency(temp))
      print("")

significantChi2('A16',categoricColumns,df2)


---------------------
A4   l    u    y
A16             
+    2  249   45
-    0  250  107
---------------------
Chisquare of A16 crosstab A4 :
Chi2 Statistic, P-value, Degree of Freedom, Expected Frequency
(21.78325079317282, 1.8613463470618034e-05, 2, array([[  0.90658499, 226.19295559,  68.90045942],
       [  1.09341501, 272.80704441,  83.09954058]]))

---------------------
A5     g  gg    p
A16              
+    249   2   45
-    250   0  107
---------------------
Chisquare of A16 crosstab A5 :
Chi2 Statistic, P-value, Degree of Freedom, Expected Frequency
(21.78325079317282, 1.8613463470618034e-05, 2, array([[226.19295559,   0.90658499,  68.90045942],
       [272.80704441,   1.09341501,  83.09954058]]))

---------------------
A6   aa   c  cc   d   e  ff   i  j   k   m   q  r   w   x
A16                                                      
+    19  60  29   7  14   7  14  3  13  16  49  2  33  30
-    33  73  11  19  10  43  41  7  35  22  26  1  30   6
---------------------
Chis

In [46]:
def nonSignificantChi2(oneColumn,columns,df):
  '''
  For p-value > 0.05, print Pearson chisquare test results of 1 column cross with 
  all other categoric columns from chosen columns
  '''
  for i in columns:
    temp=pd.crosstab(df[oneColumn], df[i])
    if stats.chi2_contingency(temp)[1] > .05:
      print("---------------------")
      print(temp)
      print("---------------------")
      print("Chisquare of A16 crosstab", i,":")
      print("Chi2 Statistic,", "P-value,", "Degree of Freedom,", "Expected Frequency")
      print(stats.chi2_contingency(temp))
      print("")
nonSignificantChi2('A16',categoricColumns,df2)

---------------------
A1     a    b
A16          
+     95  201
-    108  249
---------------------
Chisquare of A16 crosstab A1 :
Chi2 Statistic, P-value, Degree of Freedom, Expected Frequency
(0.17764082160253514, 0.6734085695133722, 1, array([[ 92.01837672, 203.98162328],
       [110.98162328, 246.01837672]]))

---------------------
A12    f    t
A16          
+    151  145
-    200  157
---------------------
Chisquare of A16 crosstab A12 :
Chi2 Statistic, P-value, Degree of Freedom, Expected Frequency
(1.4379377134356208, 0.23047335495661603, 1, array([[159.10566616, 136.89433384],
       [191.89433384, 165.10566616]]))



## Part 3 - Analysis and Interpretation

Now that you've looked at the data, answer the following questions:

- Interpret and explain the two t-tests you ran - what do they tell you about the relationships between the continuous features you selected and the class labels?
- Interpret and explain the two Chi-squared tests you ran - what do they tell you about the relationships between the categorical features you selected and the class labels?
- What was the most challenging part of this sprint challenge?

Answer with text, but feel free to intersperse example code/results or refer to it from earlier.

###t-test 
The 2 samples ttests with nonequal sample sizes  were used to see  how the features are different when grouped by the two class labels.  All tests return p-value < 0.05 so we can reject the null hypothesis, which originally stated that the means of compared samples are equals.  The most extreme p-value came from column A11 was 1.702907212870187e-22 with statistic value of 10.4894892779921.  We can interpret that mean of distribution for A11 is  much higher in the "+" class than the "-" class.

###Chi-Square
The Chi-Square tests were used on crosstab of A16 and all other categorical features to see if the two samples are independent.  All other tests except for crosstab table between A16 with A1 and A16 with A12 were significant with p-value < 0.05.  This means that we can reject the notion that they are independent.  As for Chi-Square result from A16 with A1 and A16 with A12, we would still keep the null hypothesis.  Since crosstab of A16 and A1 yields the highest p-value and lowest statistic value, we can interpret that they have the lowest significant association between the samples.

###Challenging Part
The most challenging part is making sure the tests return the correct results and analyze everything to make the correct interpretations.